# Prepare data for Russian->English search

Similar to what I did in ```04heroku_small_glossary.ipynb``` and ```05key_phrases.ipynb```

In [ ]:
import os

In [42]:
PROC_DATA_PREFIX = '/Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data'

In [43]:
!wc -l {PROC_DATA_PREFIX}/lex_preproc40_cleaned

   56460 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/lex_preproc40_cleaned


In [46]:
# Build rus-eng dictionary
ru_en_dict = dict()  # {'term': ['val1', 'val2', 'val3'], ...}
with open(os.path.join(PROC_DATA_PREFIX, 'lex_preproc40_cleaned'), 'r', encoding='utf8') as inF:
    for line in inF:
        trg, src = line.split('\t')  # swap src and trg
        try:
            ru_en_dict[src.strip()].append(trg.strip())
        except KeyError:
             ru_en_dict[src.strip()] = [trg.strip()]

print(len(ru_en_dict))

19614


In [50]:
ru_en_dict['объединенных']

['nations', 'united']

In [59]:
# Collect vocabulary sets for small datasets
import string
import nltk

stopwords = ['the', 'a', 'an', 'of',
#              'to',
#              's', 'and', 'и', 'or', 'или',
#              'been', 'being', 'by'
            ]

def preprocess(text):
    """Return a string cleaned up."""
    # lowercase
    text = text.lower()
    # word-tokenize & remove numbers if the entire token consists of numbers
    text = ' '.join(t for t in nltk.wordpunct_tokenize(text) if not t.isdigit() and not t in stopwords)
    # remove punctuation
    punct_remove = set(string.punctuation) | {'−', '\t', '\n', '\r', '\x0b', '\x0c', '◦', '°'}
    text = ''.join(char for char in text if char not in punct_remove)
    # strip extra whitespaces
    text = ' '.join(text.split())
    return text

voc_set_src = set()
voc_set_trg = set()
with open(PROC_DATA_PREFIX + '/' + 'en_ru_heroku_1000', 'r', encoding='utf8') as inF:
    for line in inF:
        line_list = line.split('\t')
        src_str, trg_str = preprocess(line_list[0].strip()), preprocess(line_list[1].strip())
        src, trg = src_str.split(), trg_str.split()
        for tok in src:
            voc_set_src.add(tok)
        for tok in trg:
            voc_set_trg.add(tok)
print('Unique words in SRC and TRG:')            
print(len(voc_set_src))
print(len(voc_set_trg))

Unique words in SRC and TRG:
2452
3638


In [54]:
# Collect the most important words

# Get a list of tok-ed TRG sentences from the raw text file
corpus_trg = []
with open(PROC_DATA_PREFIX + '/' + 'en_ru_heroku_1000', 'r', encoding='utf8') as inF:
    for line in inF:
        _, trg = line.split('\t')
        corpus_trg.append(preprocess(trg).strip())
print(len(corpus_trg))

994


In [58]:
# Score vocab items (in iterations, starting from 1000 features and going up)
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.7,  # automatically detect and filter stop words based on intra corpus document frequency of terms
    sublinear_tf=True,
    max_features=2300,  # Keep increasing until the target of 9000 lines in Glossary is reached
)
tfidf_vectorizer.fit_transform(corpus_trg)
feature_names = tfidf_vectorizer.get_feature_names()

print('Nuber of features in TRG corpus:\n', len(feature_names))
print('First 50 features:\n', feature_names[:50])

Nuber of features in TRG corpus:
 2300
First 50 features:
 ['55c', 'ii', 'iii', 'августа', 'авиационные', 'адвоката', 'администрации', 'адмирал', 'актах', 'активно', 'актов', 'акты', 'али', 'аль', 'америки', 'амисом', 'анализировать', 'английский', 'антонетти', 'антуан', 'апелляционной', 'аппараты', 'апреля', 'арабская', 'арабской', 'аспектов', 'аспекты', 'афганистана', 'афганистане', 'африканского', 'баллистическим', 'баллистических', 'банк', 'бахманьяр', 'беженцев', 'без', 'безопасного', 'безопасности', 'безопасность', 'беспилотные', 'бисау', 'ближайших', 'боевых', 'боеприпасы', 'более', 'борьбе', 'боснии', 'бригадный', 'будет', 'будут']


In [60]:
# Based on the large glossary, pick only those TRG terms that are in feature names. At the same
# time, SRC terms must occur in the small corpus to be eligibile for inclusion 
print('current number of important words:\n', len(feature_names))
heroku_glossary = []
for trg in feature_names:
    try:
        src_list = ru_en_dict[trg]
    except KeyError:
        pass
    else:
        for src in src_list:
            if src in voc_set_src:
                heroku_glossary.append('{}\t{}\n'.format(trg, src))

print('resulting heroku glossary size in lines:\n', len(heroku_glossary))
heroku_glossary[:5]

current number of important words:
 2300
resulting heroku glossary size in lines:
 6430


['ii\tannex\n',
 'августа\taugust\n',
 'авиационные\taero\n',
 'авиационные\tair\n',
 'авиационные\tcapabilities\n']

In [61]:
# Write Heroku glossary to file
with open(PROC_DATA_PREFIX + '/' + 'heroku_glossary_trg', 'w', encoding='utf8') as toF:
    for line in heroku_glossary: 
        toF.write(line)
!wc -l {PROC_DATA_PREFIX}/heroku_glossary_trg

    6430 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/heroku_glossary_trg


# Key Phrases for AutoComplete

In [63]:
from collections import Counter

In [67]:
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

def corpus_ngram_counter(corpus, n):
    """Given a list of tok-ed sentences and n for # of grams, return a counter."""
    word_counter = Counter()
    for text in corpus:
        text = text.split()
        word_counter.update(ngrammer(text, n))
    return word_counter

In [68]:
word_counter_2n = corpus_ngram_counter(corpus_trg, 2)
word_counter_3n = corpus_ngram_counter(corpus_trg, 3)
word_counter_4n = corpus_ngram_counter(corpus_trg, 4)

In [72]:
print(word_counter_2n.most_common(30), '\n')
print(word_counter_3n.most_common(30), '\n')
print(word_counter_3n.most_common(30))

[('объединенных наций', 39), ('организации объединенных', 37), ('генерального секретаря', 30), ('в отношении', 15), ('в пункте', 14), ('постановляет продлить', 13), ('мая года', 13), ('совета безопасности', 13), ('в соответствии', 12), ('г н', 12), ('соответствии с', 11), ('гуманитарной помощи', 10), ('о своей', 10), ('специального трибунала', 9), ('декабря года', 9), ('постановляет что', 9), ('просит генерального', 9), ('постановляет продолжать', 9), ('в целях', 8), ('настоящей резолюции', 8), ('связи с', 8), ('безопасности и', 8), ('прав человека', 8), ('от мая', 8), ('года s', 8), ('заниматься этим', 8), ('этим вопросом', 8), ('июня года', 8), ('международного трибунала', 8), ('наций в', 7)] 

[('организации объединенных наций', 36), ('в соответствии с', 11), ('просит генерального секретаря', 9), ('от мая года', 8), ('заниматься этим вопросом', 8), ('объединенных наций в', 7), ('соглашения о прекращении', 7), ('в связи с', 7), ('оказанию гуманитарной помощи', 7), ('постановляет прод

In [77]:
# Collect 500 unigrams
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.7,  # automatically detect and filter stop words based on intra corpus document frequency of terms
    sublinear_tf=True,
    max_features=500,
)
tfidf_vectorizer.fit_transform(corpus_trg)
feature_names = tfidf_vectorizer.get_feature_names()

print('Number of features in TRG corpus:\n', len(feature_names))
print('First 50 features:\n', feature_names[:50])

Nuber of features in TRG corpus:
 500
First 50 features:
 ['ii', 'августа', 'администрации', 'аль', 'америки', 'аппараты', 'апреля', 'арабская', 'африканского', 'беженцев', 'безопасности', 'безопасность', 'более', 'будет', 'будут', 'будучи', 'было', 'быть', 'важность', 'ваоонвс', 'введенные', 'вклад', 'включая', 'власти', 'властям', 'внимание', 'вновь', 'во', 'военные', 'военных', 'войска', 'вооруженных', 'вопроса', 'вопросам', 'вопросом', 'восточной', 'время', 'все', 'всеми', 'всех', 'выборов', 'выполняет', 'выполнять', 'выражая', 'высоко', 'выше', 'гаити', 'генерал', 'генерального', 'генеральным']


In [78]:
# Build a list of 4-, 3-, 2-, and 1-grams 
autocomplete_list = []
for ngram in word_counter_4n.most_common(30):
    autocomplete_list.append(ngram[0])

for ngram in word_counter_3n.most_common(50):
    autocomplete_list.append(ngram[0])

for ngram in word_counter_2n.most_common(100):
    autocomplete_list.append(ngram[0])

for ngram in feature_names:
    autocomplete_list.append(ngram)

print(autocomplete_list[:10])
print(autocomplete_list[-10:])

['организации объединенных наций в', 'постановляет продолжать заниматься этим', 'продолжать заниматься этим вопросом', 'соглашения о прекращении огня', 'содействие оказанию гуманитарной помощи', 'в собственности или под', 'собственности или под контролем', 'эмбарго на поставки оружия', 'о прекращении огня и', 'прекращении огня и разъединении']
['этих', 'это', 'этого', 'этой', 'этому', 'юнмовик', 'юрисдикция', 'является', 'являются', 'января']


In [80]:
# Write to file
with open(PROC_DATA_PREFIX + '/' + 'auto_complete_rus', 'w', encoding='utf8') as toF:
    toF.write('\n'.join(autocomplete_list) + '\n')
    
    
!wc -l {PROC_DATA_PREFIX}/auto_complete_rus
!head -2 {PROC_DATA_PREFIX}/auto_complete_rus
!tail -2 {PROC_DATA_PREFIX}/auto_complete_rus

     680 /Users/alexskrn/Documents/NLP/WordAlign/wordalign_notebooks/data/auto_complete_rus
организации объединенных наций в
постановляет продолжать заниматься этим
являются
января
